# **Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_distinto = '/content/drive/My Drive/Colab/files/v-2/item_nfe_classe_distinto.csv'
file_preprocessado = '/content/drive/My Drive/Colab/files/v-2/df_preprocessado.csv'
file_vocabulario = '/content/drive/My Drive/Colab/files/v-2/df_vocabulario.csv'
file_encoder = '/content/drive/My Drive/Colab/files/v-2/onehot_encoder.pkl'
file_vectorizer = '/content/drive/My Drive/Colab/files/v-2/tfidf_vectorizer.pkl'

Mounted at /content/drive


In [ ]:
!pip install unidecode
from unidecode import unidecode
import pandas as pd
import joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv(file_distinto)
len(df)

1911

In [ ]:
# Contar total de registros
total_registros = df['categoria'].count()

# Contar ocorrências específicas
percentual_CB = (df['categoria'].value_counts().get('CB', 0) / total_registros) * 100
percentual_XX = (df['categoria'].value_counts().get('XX', 0) / total_registros) * 100

# Exibir os percentuais
print(f"Percentual de CB: {percentual_CB:.2f}%")
print(f"Percentual de XX: {percentual_XX:.2f}%")

Percentual de CB: 42.07%
Percentual de XX: 57.93%


In [ ]:
df['categoria'].value_counts()

,count
categoria,
XX,1107
CB,804


In [ ]:
df['cod_unidade'].value_counts()

,count
cod_unidade,
un,1194
lt,615
pc,73
0,13
hr,10
gl,5
dl,1


In [ ]:
df['cod_anp'].value_counts()

,count
cod_anp,
0,1434
820101034,106
620505001,88
320102001,76
320102002,74
820101033,43
810101001,32
740101006,15
620501001,14


In [ ]:
df['cod_ncm'].value_counts()

,count
cod_ncm,
0,328
27101932,259
27101921,236
27101259,220
84212300,118
84219999,87
22011000,80
31021010,74
34029039,41


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Concatenar as colunas textuais com um separador
df['txt_descricao'] = df['des_anp'] + " | " + df['des_produto']

# Vetoriza a coluna textual unificada com o filtro de frequência mínima de 10
tfidf_vectorizer = TfidfVectorizer(min_df=10)
tfidf_txt_descricao = tfidf_vectorizer.fit_transform(df['txt_descricao'])

# Salvar o TfidfVectorizer para uso em produção
joblib.dump(tfidf_vectorizer, file_vectorizer)

# Obter o vocabulário gerado
vocabulario = tfidf_vectorizer.get_feature_names_out()

# Ver o tamanho do vocabulário
#print(f"\nNúmero de termos no vocabulário: {len(vocabulario)}")

# Criando o vocabulário com o índice e os termos
df_vocabulario = pd.DataFrame(vocabulario, columns=['Termo'])

# Salva o DataFrame vocabulário em um arquivo CSV
df_vocabulario.to_csv(file_vocabulario, index=False)

# Exibindo o resultado
#print(df_vocabulario)

len(df_vocabulario)

128

In [ ]:
# Codifica as variáveis categóricas
encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(df[['cod_ncm', 'cod_anp', 'cod_unidade']])

# Salvar o encoder para uso em produção
joblib.dump(encoder, file_encoder)

['/content/drive/My Drive/Colab/files/v-2/onehot_encoder0.pkl']

In [ ]:
# Cria dataframe com as variáveis categóricas codificadas e as variáveis textuais vetorizadas
X = pd.concat([
    pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(['cod_ncm', 'cod_anp', 'cod_unidade'])),
    pd.DataFrame(tfidf_txt_descricao.toarray())
], axis=1)

# Configura nomes das colunas como strings
X.columns = X.columns.astype(str)

# Cria serie com a variável alvo
y = df['categoria'] # Variável alvo

len(X), len(y)

(1911, 1911)

In [ ]:
# Cria um DataFrame com X e y
df_preprocessado = pd.concat([X, y], axis=1)

# Exibe as primeiras linhas do DataFrame combinado
#print(df_final.head())

# Salva o DataFrame combinado em um arquivo CSV
df_preprocessado.to_csv(file_preprocessado, index=False)

In [ ]:
len(df_preprocessado)

1911

In [ ]:
len(df_preprocessado.columns)

184

In [ ]:
#df_preprocessado.info()
pd.set_option('display.max_columns', 30)
colunas = df_preprocessado.columns.tolist()
print(colunas)
print("-" * 50)
print(df_preprocessado.dtypes)
print("-" * 50)
print(df_preprocessado.dtypes.head(30))

['cod_ncm_0', 'cod_ncm_17049020', 'cod_ncm_18069000', 'cod_ncm_19053200', 'cod_ncm_19059090', 'cod_ncm_21050010', 'cod_ncm_21069050', 'cod_ncm_22011000', 'cod_ncm_22021000', 'cod_ncm_22029900', 'cod_ncm_22071090', 'cod_ncm_22072019', 'cod_ncm_27101259', 'cod_ncm_27101921', 'cod_ncm_27101932', 'cod_ncm_27101999', 'cod_ncm_31021010', 'cod_ncm_33074900', 'cod_ncm_34023990', 'cod_ncm_34029039', 'cod_ncm_34029090', 'cod_ncm_38111900', 'cod_ncm_38112120', 'cod_ncm_38119010', 'cod_ncm_38119090', 'cod_ncm_38190000', 'cod_ncm_38249941', 'cod_ncm_56012291', 'cod_ncm_76169900', 'cod_ncm_84212300', 'cod_ncm_84212990', 'cod_ncm_84213100', 'cod_ncm_84213990', 'cod_ncm_84219999', 'cod_ncm_85129000', 'cod_ncm_87089990', 'cod_anp_0', 'cod_anp_320101001', 'cod_anp_320102001', 'cod_anp_320102002', 'cod_anp_620501001', 'cod_anp_620505001', 'cod_anp_740101003', 'cod_anp_740101006', 'cod_anp_810101001', 'cod_anp_820101012', 'cod_anp_820101033', 'cod_anp_820101034', 'cod_unidade_0', 'cod_unidade_dl', 'cod_un